<font size = "6"> Receipt Reprint Template <font>

<font size = "5">Summary <font> 

<font size = "4">This notebook is a template and guideline for reprinting customer receipts. The below code and accompanying word doc template should allow you to recreate virtually any receipt and letter required to be sent to customers. <font>

<font size = "5">Before reading or adding to this document please save the document to your local computer or another location to avoid tampering with or changing this document. <font>

<font size = '4'>Libraries required for this notebook: <font>

In [ ]:
#Not all libraries may be required for the reprint process but these should include all libraries
#required for analysis and visualization of the data is required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pyodbc
import datetime
import seaborn as sns
from scipy import stats
from functools import reduce
from matplotlib.pyplot import figure
pd.set_option('display.max_columns', 500)

<font size = '4'>In order to directly query WU datawarehouses and run the SQL query's included in this notbook you will need to connect the notebook with ODBC using the method below. If you do not know your ODBC data source name (the name of the datawarehouse you would like to query) you can search ODBC on your computer which should bring up the ODBC Data Source Administrator which includes all the names of datasources your computer can directly connect to. <font>

In [ ]:
# Connect with ODBC connection to directly query Netezza darawarehouse 
#If you do not know your DSN name or are having trouble you can query the databases
#via Aginity and save the resulting data to a csv which you can then upload to this notebook
#via the pandas read_csv method
dsn_name = "DSN = ENTER YOUR COMPUTERS DSN NAME HERE;"
conn = pyodbc.connect(dsn_name)

<font size = '4'>Below is an example of a SQL query used to identify affected customer transactions which may require remedication. The queries MTCN's can then be used as an input for a larger SQL query which will pull all fields required for a receipt reprint. Notice how the formatted (f string) string method is used for this query (and all SQL queries in this notebook), if you are receiving errors when running a SQL query in this notebook which works in another SQL workbench (Aginity) check to ensure your SQL query is saved as a formatted string. <font> 

In [ ]:
#All transactions sent from Rite Aid locations between 09-30-2017 and 03-28-2018
#require remediation if the transactions have not yet been picked up
#below query identifies the current population
rite_aid_sql = f'''
SELECT tran.MTCN03 || tran.MTCN07 AS MTCN, fin.*  
FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 fin
LEFT  JOIN TDB.ETL_TDB.DB100PDLIB_DBAGTP01 agt ON fin.RECAGTKEY = agt.AGTKEY
LEFT  JOIN TDB.ETL_TDB.DW400PDLIB_DWTXKP01 tran ON fin.TXNKEY = tran.TXNKEY
WHERE fin.RECISODTE between '2017-09-30' and '2018-03-28' 
	and fin.RCONTRYKEY = 222 
	and fin.PCONTRYKEY = 0 
	and fin.ICONTRYKEY != 222
	and UPPER(agt.AGTNME) like '%RITE AID%'
'''

<font size = '4'> The below line will execute your SQL query and save the query to a notebook which can then be further manipulated <font>

In [ ]:
#Execute above SQL statement
rite_aid = pd.read_sql_query(rite_aid_sql,conn)

<font size = '4'>The below line will extract the MTCN's from the affected transaction and create a formatted string of MTCN's which can be used in a SQL query. Note if your above query only identifies each transaction by its transaction key (TXNKEY) you will need to add a step to your query to join the TXNKEY with the MTCN in another table. The query used in this notebook to obtain the required customer data for a receipt reprint requires transaction MTCN's as the input rather than transaction key's. <title>

In [ ]:
#Convert the list of MTCN's into string which can be used in a SQL statement 
mtcn = [str(i) for i in rite_aid.MTCN]
mtcn = "','".join(mtcn)
mtcn = "'" + mtcn + "'"

<font size = '4'>The below SQL queries will pull the bulk of the required data needed for a receipt reprint and takes the mtcn variable above as an input near the bottom of the query. If your query is showing an error check and make sure the mtcn variable is being appropriately called. <font>

In [ ]:
#Extract all data needed to create upload file
upload_file_dataframe = """
SELECT 
   -- If an MTCN is found in Manual Refunds, Auto Refunds, or Cancelled Transactions tables, return TRUE, else FALSE
   NVL2( ManualRefunds.MTCN , 'ManualRefund', NVL2( AutoRefunds.MTCN , 'AutoRefund', NVL2( CancelledTransactions.MTCN , 'Cancelled', 'No'))) AS RefundedOrCancelled,
   -- Select all fields from the inner sub-query (only tables from TDB)
   TdbInfo.*,
  
   -- Customer Name, various sources, use what looks best
   CopsMedallia.SENDER_NAME,
   CopsTransactions.SENDNAME,
   CopsMedallia.SENDER_FIRST_NAME,
   CopsTransactions.SENDERFIRSTNAME,   
   CopsMedallia.SENDER_MIDDLE_NAME,
   CopsTransactions.SENDERMIDDLENAME,
   CopsMedallia.SENDER_MATERNAL_LAST_NAME,
   CopsTransactions.SENDERMATERNALLASTNAME,
   CopsMedallia.SENDER_PATERNAL_LAST_NAME,
   CopsTransactions.SENDERPATERNALLASTNAME,
   
   -- Email, use what looks best
   CopsMedallia.SENDER_EMAIL,
   CopsTransactions.SENDEREMAILADDRESS,
   
   
   -- Customer Address
   CopsMedallia.SENDER_STRADR1,
   CopsMedallia.SENDER_STRADR2,
   CopsMedallia.SENDER_CITY,
   CopsTransactions.SENDERCITYNAME,
   CopsMedallia.SENDER_STATE,
   CopsTransactions.SENDERSTATENAME,
   CopsMedallia.SENDER_ZIPCDE,
   CopsTransactions.SENDERPOSTALCODE,
   CopsMedallia.SENDER_CONTRY,
   CopsTransactions.SENDERCOUNTRY,
   CopsTransactions.SENDCOUNTRYOFRESIDENCE,
   CopsTransactions.SENDERTELEPHONE,
   
-- Misc
   CopsTransactions.TRANSACTIONBRANDIDENTIFIER,
   CopsMedallia.LEGAL_ENTITY,
-- Only populated for US outbound > $15
   DfBuffer.TOTTOREC,
   DfBuffer.DATEAVAIL
   

FROM (
    SELECT 
        MTCN03 || MTCN07 AS MTCN,
        FinTransactions.TXNKEY,
    
    -- PRODUCTS, usually recording product == pay product
        FinTransactions.RECPRODCT,
        Products.PRDDES AS PRODUCT_DESCRIPTION,
        Products.BRAND,
         
    -- Dates/times
        FinTransactions.RECISODTE + SendTimes.HOURMINC AS RECISODTE,
        FinTransactions.PAYISODTE + PayoutTimes.HOURMINC AS Pay_Date,
        -- Do we need RDATE7?
        TransactionDetails.RDATE7,
    
    -- Principal amounts
        HistoricalRecordingFacts.RECPRN,
        RecordingCurrencies.CURR AS Send_Currency,
        PayoutDetails.PAYPRN AS Pay_Amount,
        PayoutCurrencies.CURR AS Pay_Currency,
        
    -- Tax and Fees
        HistoricalRecordingFacts.CHGBAS AS Send_Fee,
        HistoricalRecordingFacts.CHGBASUSD AS Send_Fee_USD,
        HistoricalRecordingFacts.TAXCITY, 
        HistoricalRecordingFacts.TAXST,
        HistoricalRecordingFacts.TAXMUN, 
        HistoricalRecordingFacts.TAXTTL,
    
    -- Fx And Dsc
        TransactionDetails.RATERECEXC / 100 as RATERECEXC100,
        HistoricalRecordingFacts.RECDSC / 100 as RECDSC100,
        TransactionDetails.RATEPAYEXC / 100 as RATEPAYEXC100,
        PayoutDetails.PAYDSC / 100 as PAYDSC100,
        -- Do we use this??
        HistoricalRecordingFacts.AMTDSC / 100 as AMTDSC100,
        
    -- Locations (Agent based not customer address based)    
        RecordingAgents.CONTRY AS Origination_Country,
        IntendedCountries.CONTRY AS Intended_Country,
        PayoutCountries.CONTRY AS Pay_Country,
        
    -- Sending/Recording Agent info
        RecordingAgents.AGTNME AS Recording_Agent_Name,
        -- Change "sending" to "recording" for consistency
        RecordingAgents.AGNTID AS Sending_Agent_ID,
        RecordingAgents.ACITY AS Sending_Agent_City,
        RecordingAgents.NETCONTRY AS Recording_Network_Country,
        RecordingAgents.NETAGTNME AS Recording_Network_Name,
        RecordingAgents.NAID AS Recording_Network_Id,

    -- Paying Agent info   
        PayingAgents.AGTNME AS Paying_Agent_Name,
        PayingAgents.AGNTID AS Paying_Agent_ID,
        PayingAgents.ACITY AS Paying_Agent_City,
        PayingAgents.NETCONTRY AS Paying_Network_Country,
        PayingAgents.NETAGTNME AS Paying_Network_Name,
        PayingAgents.NAID AS Paying_Network_Id,

    -- Misc
        TransactionDetails.PRMCDE

    FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 FinTransactions
        
        INNER JOIN TDB.ETL_TDB.DB100PDLIB_FNPRDP01 Products on FinTransactions.PRDKEY = Products.PRDKEY 
        INNER JOIN TDB.ETL_TDB.DW400PDLIB_DWTXKP01 TransactionDetails ON FinTransactions.TXNKEY = TransactionDetails.TXNKEY
        INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBTMSP01 SendTimes ON FinTransactions.RECTMSKEY = SendTimes.TMSKEY
        INNER JOIN TDB.ETL_TDB.DW200PDLIB_DWRTXP01 HistoricalRecordingFacts ON FinTransactions.TXNKEY = HistoricalRecordingFacts.TXNKEY
        INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBCURP01 RecordingCurrencies ON FinTransactions.RECCURKEY = RecordingCurrencies.CURKEY
        INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBCTYP01 IntendedCountries ON FinTransactions.ICONTRYKEY = IntendedCountries.CONTRYKEY
        INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBCTYP01 RecordingCountries ON FinTransactions.RCONTRYKEY = RecordingCountries.CONTRYKEY
        
        -- Recording agent key
        INNER JOIN (
            SELECT * FROM TDB.ETL_TDB.DB100PDLIB_DBAGTP01 A
            LEFT OUTER JOIN TDB.ETL_TDB.DB100PDLIB_DBNWKP01 B ON A.NWKKEY = B.NWKKEY
        ) as RecordingAgents ON FinTransactions.RECAGTKEY = RecordingAgents.AGTKEY
        
        -- Anything Payout must be left joined, unless you want to exclude anything not yet paid out
        LEFT OUTER JOIN TDB.ETL_TDB.DB100PDLIB_DBTMSP01 PayoutTimes ON FinTransactions.PAYTMSKEY = PayoutTimes.TMSKEY
        LEFT OUTER JOIN TDB.ETL_TDB.DW300PDLIB_DWPTXP01 PayoutDetails ON FinTransactions.TXNKEY = PayoutDetails.TXNKEY
        LEFT OUTER JOIN TDB.ETL_TDB.DB100PDLIB_DBCURP01 PayoutCurrencies ON FinTransactions.PAYCURKEY = PayoutCurrencies.CURKEY
        LEFT OUTER JOIN TDB.ETL_TDB.DB100PDLIB_DBCTYP01 PayoutCountries ON FinTransactions.PCONTRYKEY = PayoutCountries.CONTRYKEY
        
        -- Paying agent key
        LEFT OUTER JOIN ( 
            SELECT * FROM TDB.ETL_TDB.DB100PDLIB_DBAGTP01 A
            LEFT OUTER JOIN TDB.ETL_TDB.DB100PDLIB_DBNWKP01 B ON A.NWKKEY = B.NWKKEY
        ) as PayingAgents ON FinTransactions.PAYAGTKEY = PayingAgents.AGTKEY

        WHERE 
        FINtransactions.recisodte > '2016-06-01'
        AND MTCN in ({mtcn}) --The mtcn variable here is being called from the variable you created previous to this query, make sure you do not change the formatting here or you 
        

) TdbInfo

LEFT OUTER JOIN COPS.DBAUSER.TRANSACTIONS CopsTransactions ON TdbInfo.MTCN = CopsTransactions.MTCN AND Substr(TdbInfo.RECISODTE,1,10) = Substr(CopsTransactions.RECORDINGDATETIME,1,10)
LEFT OUTER JOIN COPS.DBAUSER.MTCN_PRODUCT_MEDALLIA CopsMedallia ON TdbInfo.TXNKEY = CopsMedallia.TXNKEY
LEFT OUTER JOIN COPS.DBAUSER.CFPB_DF_BUFFER DfBuffer on TdbInfo.TXNKEY = DfBuffer.TXNKEY

-- These are the (incomplete) refund tables. We use them to indicate refunded txns, but missing values are common.
LEFT OUTER JOIN COPS.DBAUSER.MTCN_MANUAL_REFUND ManualRefunds ON TdbInfo.MTCN = ManualRefunds.MTCN AND Substr(TdbInfo.RECISODTE,1,10) = Substr(ManualRefunds.REC_DATETIME,1,10)
LEFT OUTER JOIN COPS.DBAUSER.MTCN_AUTO_REFUND AutoRefunds ON TdbInfo.MTCN = AutoRefunds.MTCN AND Substr(TdbInfo.RECISODTE,1,10) = Substr(AutoRefunds.REC_DATETIME,1,10)
LEFT OUTER JOIN COPS.DBAUSER.MTCN_CANCEL_TXNS CancelledTransactions ON TdbInfo.MTCN = CancelledTransactions.MTCN AND Substr(TdbInfo.RECISODTE,1,10) = Substr(CancelledTransactions.REC_DATETIME,1,10)
"""

In [ ]:
#SQL query to identify customers who have not picked up transaction with bad receipts and identify customer information
#required to mail a complete receipt to the customer
#these queries are used to create the customer receipts rather than the upload file
receipt_reprint_dataframe = """
SELECT  riteaid.MTCN, riteaid.TXNKEY, med.SENDER_NAME, riteaid.BUSPRDGRP, riteaid.principal_usd, 
		riteaid.CHGBASUSD, riteaid.AMTDSC, riteaid.AMTGROSS, riteaid.convert_local_currency,
		riteaid.pay_amount_local_currency,riteaid.RECISODTE, med.RECORDING_DATE, 
		med.RECORDING_TIME, med.SENDER_EMAIL, med.SENDER_STRADR1, med.SENDER_CITY, 
		med.SENDER_STATE, med.SENDER_CONTRY, med.PAYER_NAME, currency.RECORDING_CURRENCY, 
		med.INTENDED_COUNTRY, currency.INTENDED_CURRENCY, med.SENDER_CTYDES, med.SENDER_ZIPCDE, 
		riteaid.AGTKEY, riteaid.AGNTID,riteaid.CSBPHONE1, riteaid.CSBURL1,riteaid.STAGCNME, 
		riteaid.DATEAVAIL, riteaid.AGTNME, riteaid.AADDR1, riteaid.ACITY, riteaid.ASTATE, 
		riteaid.ZIPCDE, riteaid.CONTRY
		
FROM COPS.DBAUSER.MTCN_PRODUCT_MEDALLIA med
INNER JOIN(
SELECT fin.TXNKEY, fin.RECAGTKEY, fin.RECISODTE, agents.AGTKEY,prod.BUSPRDGRP, 
agents.AGNTID, tran.MTCN03||tran.MTCN07 AS MTCN, hist.RCPUSD AS principal_usd, hist.RECDSC, 
hist.CHGBASUSD, hist.AMTDSC, hist.AMTGROSS, buff.CSBPHONE1, buff.CSBURL1, 
tran.RATERECEXC/100*tran.RATEPAYEXC/100 AS convert_local_currency,convert_local_currency*hist.AMTGROSS AS pay_amount_local_currency, 
buff.STAGCNME, buff.DATEAVAIL, agents.AGTNME, agents.AADDR1, agents.ACITY, 
agents.ASTATE, agents.ZIPCDE, agents.CONTRY
FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 fin

INNER JOIN TDB.ETL_TDB.DB100PDLIB_DBAGTP01 agents ON fin.RECAGTKEY = agents.AGTKEY            
INNER JOIN TDB.ETL_TDB.DB100PDLIB_FNPRDP01 prod ON fin.RECPRDKEY = prod.PRDKEY
INNER JOIN TDB.ETL_TDB.DW400PDLIB_DWTXKP01 tran ON fin.TXNKEY = tran.TXNKEY
INNER JOIN TDB.ETL_TDB.DW200PDLIB_DWRTXP01 hist ON fin.TXNKEY = hist.TXNKEY
INNER JOIN COPS.DBAUSER.CFPB_DF_BUFFER buff ON fin.TXNKEY = buff.TXNKEY

WHERE fin.RECISODTE BETWEEN '2017-09-30' AND '2018-03-28'
    AND fin.RCONTRYKEY = 222
    -- Filter out unpaid/cancelled txns and domestic transctions
    AND fin.PCONTRYKEY = 0
    AND fin.ICONTRYKEY != 222
    -- Only Rite Aid recorded transactions
    AND UPPER(Agents.AGTNME) LIKE '%RITE AID%') as riteaid ON med.MTCN = riteaid.MTCN
	
INNER JOIN(
SELECT tran.TXNKEY, icurr.CURR AS INTENDED_CURRENCY, rcurr.CURR AS RECORDING_CURRENCY
FROM TDB.ETL_TDB.DB100PDLIB_FNTXKP01 tran
LEFT JOIN TDB.ETL_TDB.DB100PDLIB_DBCURP01 icurr ON tran.INTCURKEY = icurr.CURKEY  
LEFT JOIN TDB.ETL_TDB.DB100PDLIB_DBCURP01 rcurr ON tran.RECCURKEY = rcurr.CURKEY) as currency ON riteaid.TXNKEY = currency.TXNKEY

WHERE med.RECORDING_DATE BETWEEN '2017-06-30' AND '2018-03-28'
"""

In [ ]:
#Two final data points are required to complete the information for the receipt
#iCare case number will be obtained after providing upload file and cases are created
#for each receipt. Recording operator ID numbers are also required, the below query searches for the 
#recording operator ID 
recording_operator_id_sql_query = f"""
SELECT * FROM COPS.DBAUSER.MTCN_ACTIVITY act
WHERE act.ACTTYP = 'REC' 
AND act.ACTDTE > '2016-12-01'
AND MTCN in ({mtcn})
"""

<font size = '4'>The below line will execute the above SQL query. After executing the query it is recomended you export the data frame to either a pickle or csv file on your computer in order to easily and quickly reload the data incase you have to restart this notebooks kernel and clear all outputs as the above query can take 10 - 15 minutes to execute.<font>

In [ ]:
#Execute above SQL statements
#If the raw_data_query has already been done for the day, export it 
#to either a pickle or csv and import via the read_pickle
#or read_csv pandas function to save time
upload_file_data = pd.read_sql_query(upload_file_data,conn)
receipt_reprint_data = pd.read_sql_query(receipt_reprint_dataframe,conn)
operator_id_data = pd.read_sql_query(recording_operator_id_sql_query,conn)

<font size = '4'> Below are two methods to read in pickle or csv files which is quicker than running the above queries. This method is recomended if the above queries have already been run and saved earlier or if their were issues connecting to the ODBC driver and the SQL queries were run and exported from an alternative SQL workbench. The # before the lines indicates they are currently read as comments and will not execute, in order to execute the lines remove the # and add the file paths. <font>

In [ ]:
#upload_file_data = pd.read_csv(r'exported file path', sep = '|')
#upload_file_data = pd.read_pickle](path_to_pickle_file)

In [ ]:
#Drop duplicate MTCN's, the first recording operator id
#will be kept as that was the operator who first touched
#the transaction
recording_operator_id = recording_operator_id.drop_duplicates(subset = "MTCN", keep = 'first')
receipt_reprint_data = receipt_data.drop_duplicates(subset = 'TXNKEY')
upload_file_data = recording_operator_id.drop_duplicates(subset = "MTCN", keep = 'first')

<font size = '4'> The recording operator id data frame will be merged with the receipt reprint data in order to create a single unified file which includes all information needed to create a new customer receipt (with the exception of he iCare case number which can only be obtained after uploading the upload file to Pune and case numbers are generated). <font>

In [ ]:
#Merge receipt_data and recording_operator_id
receipt_receording_merged = pd.merge(receipt_data, recording_operator_id, how = 'left', left_on = 'MTCN', right_on = 'MTCN' )

<font size = '4'>The below lines will create the mail_merge data frame which will ultimately be exported and used to create the customer receipts. If additional fields are needed they can be added below but you may need to adjust the SQL queries to include the additional inforation. <title>

In [ ]:
#Create dataframe containing only information required for mail merge
mail_merge = pd.DataFrame()
mail_merge['MTCN'] = receipt_receording_merged.MTCN
mail_merge['operator_id'] = receipt_receording_merged.OPRID
mail_merge['product'] = receipt_receording_merged.BUSPRDGRP
mail_merge['agent_name'] = receipt_receording_merged.AGTNME
mail_merge['agent_address'] = receipt_receording_merged.AADDR1
mail_merge['agent_city'] = receipt_receording_merged.ACITY
mail_merge['agent_state'] = receipt_receording_merged.ASTATE
mail_merge['agent_zip_code'] = receipt_receording_merged.ZIPCDE
mail_merge['transaction_date'] = pd.DatetimeIndex(receipt_receording_merged.RECISODTE).strftime('%B%d%Y')
mail_merge['customer_name'] = receipt_receording_merged.SENDER_NAME
mail_merge['time_of_transaction'] = pd.DatetimeIndex(receipt_receording_merged.RECORDING_TIME).strftime("%I:%M %p")
mail_merge['customer_name'] = receipt_receording_merged.SENDER_NAME
mail_merge['transaction_principal'] = round(receipt_receording_merged.PRINCIPAL_USD,2).apply(lambda row: str(row))
#mail_merge['transaction_principal'] = round(receipt_data.PRINCIPAL_USD.apply(lambda row: str(round(row,2)))                                            
mail_merge['transaction_fee'] = round(receipt_receording_merged.CHGBASUSD,2).apply(lambda row: str(row))
mail_merge['transaction_total'] = round(receipt_receording_merged.AMTGROSS,2).apply(lambda row: str(row))
mail_merge['exchange_rate'] = round(receipt_receording_merged.CONVERT_LOCAL_CURRENCY,2).apply(lambda row: str(row))
mail_merge['int_currency'] = receipt_receording_merged.INTENDED_CURRENCY
mail_merge['transaction_date'] = pd.DatetimeIndex(receipt_receording_merged.RECISODTE).strftime('%m/%d/%Y')
mail_merge['customer_name'] = receipt_receording_merged.SENDER_NAME
mail_merge['customer_adress'] = receipt_receording_merged.SENDER_STRADR1.apply(lambda row:row.title())
mail_merge['customer_city'] = receipt_receording_merged.SENDER_CITY.apply(lambda row:row.title())
mail_merge['customer_state'] = receipt_receording_merged.SENDER_STATE
mail_merge['customer_zip'] = receipt_receording_merged.SENDER_ZIPCDE
mail_merge['sender_country'] = receipt_receording_merged.SENDER_CTYDES
mail_merge['receiver_country'] = receipt_receording_merged.INTENDED_COUNTRY
mail_merge['receiver_name'] = receipt_receording_merged.PAYER_NAME
mail_merge['date_of_availability'] = receipt_receording_merged.DATEAVAIL.apply(lambda row: datetime.datetime(int(row[-4:]),int(row[:2]),int(row[2:4])).strftime('%B %d, %Y'))
mail_merge['state_financial_regulatory_body'] = receipt_receording_merged.STAGCNME
mail_merge['state_financial_regulatory_body_phone'] = receipt_receording_merged.CSBPHONE1
mail_merge['state_financial_regulatory_body_website'] = receipt_receording_merged.CSBURL1

<font size = '4'> This will export your mail_merge data frame as an excel file which will be required to perform a mail merge with the word template which accompanies this notebook. However you may export the data to a csv and use the data import function in an empty Excel file to import the data in to an excel file, this may allow greater customization of the settings of the data within the excel file (basically you can change excel column data types when importing a csv where as a direct excel export may result in certain columns having the incorrect data types). You may also receive a series of errors  about exceeding Excel's limit for URLS, these errors can be ignored and are caused by embedded links in the data frame.<font>

In [ ]:
#Export the mail_merge dataframe to an Excel doc to complete mail merge
writer = pd.ExcelWriter(r'file path.xlsx')
mail_merge.to_excel(writer, index = False)
writer.save()

<font size = '4'>The remainder of this notebook covers the creation of an upload file which needs to be created and uploaded into iCare in order to create iCare case numbers which will be included in the letter which will accompany the receipt sent to customers. In order for the case numbers to be generated the upload file needs to be formatted as specified below and sent to Pune. <font>

In [ ]:
#Copy raw data for manipulation into upload file
upload_template_df = upload_file_data.copy() 

In [ ]:
upload_template_df = upload_template_df.where((pd.notnull(upload_template_df)), None)

<font size = '4'> The below functions are required to format and merge required data for the upload file. <font>

In [ ]:
#Convert the open_transactions DF to an upload file
#Format first, middle and last name of sender

def fallback(row, col1, col2):
    if row[col1] is not None:
        return row[col1]
    elif row[col2] is not None:
        return row[col2]
    else:
        # always return at least an empty string
        return ''

def first_name(row):
    return fallback(row, 'SENDERFIRSTNAME','SENDER_FIRST_NAME')
    
def middle_name(row):
    return fallback(row, 'SENDERMIDDLENAME', 'SENDER_MIDDLE_NAME')

def maternal_name(row):
    return fallback(row, 'SENDERMATERNALLASTNAME','SENDER_MATERNAL_LAST_NAME')

def paternal_name(row):
    return fallback(row, 'SENDERPATERNALLASTNAME','SENDER_PATERNAL_LAST_NAME')

def last_name(row):
    last_name = paternal_name(row) + ' ' + maternal_name(row)
    return last_name.strip()

#Check if upload_email field has an email in it, if so return 'Yes' otherwise leave blank
def email(row):
    return fallback(row, 'SENDER_EMAIL', 'SENDEREMAILADDRESS')

#Identify customer emails if required (only digital US customers and non-US customers will have their emails retained)
def should_send_email(row):
    email_address = email(row)
    if email_address == '':
        return '' # dont send email
    elif row['SENDER_CONTRY'] !='US':
        return 'Yes' # send email
    elif row['RECPRODCT'][0] == 'F':
        return 'Yes'
    else:
        return ''

#Check both customer street fields and merge if both fields contain information
def street_merge(row):
    if row['SENDER_STRADR1'] == None or 'NaN':
        return ''
    else:
        return row['SENDER_STRADR1'].strip()

def city_check(row):
    return fallback(row,'SENDER_CITY','SENDERCITYNAME')

def state_check(row):
    return fallback(row,'SENDER_STATE', "SENDERSTATENAME")

def zipcode_check(row):
    return fallback(row, 'SENDERPOSTALCODE', 'SENDER_ZIPCDE')

def country_check(row):
    return fallback(row, 'SENDER_CONTRY', 'SENDCOUNTRYOFRESIDENCE')

<font size = '4'> The below will format your upload_templat_df and execute the above functions, certain lines/functions may need to be adjusted based on the information available. <font>

In [ ]:
#Execute above functions to prepare fields for upload file
upload_template_df['upload_zipcode'] = upload_template_df.apply(zipcode_check, axis = 1)
upload_template_df['upload_street'] = upload_template_df.apply(street_merge, axis = 1)
upload_template_df['upload_city'] = upload_template_df.apply(city_check, axis = 1)
upload_template_df['upload_state'] = upload_template_df.apply(state_check, axis = 1)
upload_template_df['sender_first_name_updated'] = upload_template_df.apply(first_name,axis = 1)
upload_template_df['sender_middle_name_updated'] = upload_template_df.apply(middle_name,axis = 1)
upload_template_df['sender_last_name_updated'] = upload_template_df.apply(last_name,axis = 1)
upload_template_df['upload_email'] = upload_template_df.apply(email, axis = 1)
upload_template_df['upload_zipcode'] = upload_template_df.apply(zipcode_check, axis = 1)
upload_template_df['upload_country'] = upload_template_df.apply(country_check, axis = 1)
upload_template_df['long_mtcn'] = upload_template_df.apply(lambda row: pd.to_datetime(row['RECISODTE'],errors = 'coerce').strftime('%y%j') + '8' + str(row.MTCN), axis = 1)
upload_template_df['Refund Amount'] = upload_template_df.SEND_FEE

<font size = '4'> The country names in iCare need to be formatted in a specific manner which does not align with the formatting in our databases. The below lines will read in a table with the appropriate iCare country formatting which will then be merged with the upload file to ensure proper formatting for the Originiation Country, Intended Country and Pay Country fields. <font>

In [ ]:
#Import iCare Country lookup codes to convert current country codes to names accepted by iCare
iCare_cc = pd.read_csv(r"C:\Users\309270\Desktop\portfolio_analytics\templates\iCare-Country-lookup.csv")

In [ ]:
#Replace country codes in final upload report with iCare country names
upload_template_df = pd.merge(upload_template_df, iCare_cc, how = 'left', left_on = 'ORIGINATION_COUNTRY', right_on = 'CONTRY')
upload_template_df = pd.merge(upload_template_df, iCare_cc, how = 'left', left_on = 'INTENDED_COUNTRY', right_on = 'CONTRY')
upload_template_df = pd.merge(upload_template_df, iCare_cc, how = 'left', left_on = 'PAY_COUNTRY', right_on = 'CONTRY')

<font size = '4'> In order to automatically assign bots to take the uploaded cases, the bots need to be added to the upload_template_df. Before running this line check the available bot id's and make sure they match the below id's otherwise the bots will not pickup the transactions. The below list of bots should be updated/changed based on the available bots. <font>

In [ ]:
# list available bot ids
remediation_bots = [ '005150000077DHh', '005150000077DIe', '005150000077DIP', '0051C000005QbW2', '0051C000005QbWH', '0051C000005QbWM' ]
# just making sure the list is at least as long as the df...
remediation_bots_iter = iter(remediation_bots * len(upload_template_df))

<font size = '4'> The below lines will finalze the upload_file and ensure appropriate formatting. Specific fields will need to be adjusted based on the specific cases:
    * The Description, REM Letter Resolution and Public Comment fields will need to be updated as they change with each case
    * Fields such as the Pay Date or Pay Amount may also need to be adjusted as they vary based on each case 
    

In [ ]:
#Create final upload file
upload_file = pd.DataFrame()
upload_file['Case Owner'] = [next(remediation_bots_iter) for i in upload_template_df.index]
upload_file['Subject'] = ['ADD CASE NUMBER AND NAME HERE']*len(upload_template_df)
upload_file['Description'] = ['']*len(upload_template_df) #Description field is to be left empty, issue description can be found in the 'REM Letter Resolution Text' field
upload_file['Status'] = ['New']*len(upload_template_df)
upload_file['Case Record Type'] = ['Remediation']*len(upload_template_df)
upload_file['REM Letter Resolution Text'] = ['ADD REM LETTER RESOLUTION LANGUAGE HERE']*len(upload_template_df)
upload_file['Preferred Language'] = ['']*len(upload_template_df) #Preferred language is to be left empyt
upload_file['Refund Amount'] = upload_template_df.apply(lambda row: round(row['Refund Amount'],2),axis = 1)
upload_file['Refund Currency'] = upload_template_df['SEND_CURRENCY']
upload_file['Remediation'] = ['Refund Fee']*len(upload_template_df)
upload_file['Public Comments'] = ['ADD PUBLIC COMMENTS HERE']*len(upload_template_df) #Manually entered information relating to the reason for the refund
upload_file['REM First Name'] = upload_template_df['sender_first_name_updated']
upload_file['REM Middle Name'] = upload_template_df['sender_middle_name_updated']
upload_file['REM Last Name'] = upload_template_df['sender_last_name_updated']
upload_file['REM Phone Number'] = upload_template_df['SENDERTELEPHONE']
upload_file['REM Email'] = upload_template_df['upload_email']
upload_file['Street'] = upload_template_df['upload_street']
upload_file['City'] = upload_template_df['upload_city']
upload_file['State'] = upload_template_df['upload_state']
upload_file['Zip/Postal Code'] = upload_template_df['upload_zipcode']
upload_file['Country'] = upload_template_df['CTYDES_x']
upload_file['Long MTCN'] = upload_template_df['long_mtcn']
upload_file['MTCN'] = upload_template_df['MTCN'] 
upload_file['Send Date'] = pd.DatetimeIndex(upload_template_df['RECISODTE']).strftime('%m/%d/%Y')
upload_file['Send Amount'] = upload_template_df.apply(lambda row: round(row['RECPRN'],2), axis = 1)
upload_file['Send Currency'] = upload_template_df['SEND_CURRENCY']
upload_file['Send Fee'] = upload_template_df['SEND_FEE']
upload_file['Origination Country'] = upload_template_df['CTYDES_x']
upload_file['Send Brand'] = upload_template_df['BRAND'] 
upload_file['Pay Date'] = "" #None of the transactions have been picked up but the bots may have a problem interpreting a Nan field
#upload_file['Pay Date'] = pd.DatetimeIndex(upload_template_df['PAY_DATE']).strftime('%m/%d/%Y')#Add logic to find any 0 values and change to Null value
upload_file['Pay Amount'] = 0 #Only individuals who have not picked up the transaction are eligible to receive receipts
upload_file['Pay Currency'] = upload_template_df['PAY_CURRENCY']  
upload_file['Destination Country'] = upload_template_df['CTYDES_y']
upload_file['Sending Agent ID'] = upload_template_df['SENDING_AGENT_ID']
upload_file['Send Email'] = upload_template_df.apply(should_send_email, axis=1)

<font size = '4'> When exporting a csv file make sure the file ends as a .txt rather than .csv, they are the same but the Pune team may push back if not formatted with a .txt file. Also ensure the sep is set to pipe deliminated which is '|' and the index is set to 'False' which  removes the index column from the exported file. <font>

In [ ]:
upload_file.to_csv(r'C:\Users\309270\Desktop\portfolio_analytics\rite_aid_receipt_issue\upload_file.txt', sep = '|',index = 'False')

<font size = '4'> Afer uploadin the upload file, review it to ensure there are no errors, in particular look at the maximum refund amount to ensure there were no errors which would result in over refunding customers. After the document is reviewed send the upload file to the Technology Engineering Center who will then upload the file into iCare and generate case numbers for each refund. <font>

<font size = '4'> After uploading the file the case numbers can be queried from the iCare database and added to the customer receipt dataframe. After adding this field you will use a mail merge to add the information in the exported excel receipt file to the Word template. If you have not performed a mail merge before search the name in the Word doc's quetsion field which will then provide a step by step guide to using a mail merge. <font>

<font size = '5'>Change Log<font>

<font size = '4'>Feb 1, 2019 -- document created by David Thomas <font>